# Part 6.1 - Prepare Data for Analysis
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import sys, gc
sys.path.insert(0, '../utils')
from utils import conf_pyspark, load_dataset
from utils_preplot import preplot, load_doctopic
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)
from pyspark.sql import functions as F
from pyspark.sql.types import *

import datetime
print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# intiate PySpark
sc, spark = conf_pyspark()

sc

2019-02-03 21:13:53
[('spark.driver.port', '46852'),
 ('spark.driver.host', 'x99.hub'),
 ('spark.app.id', 'local-1549181636142'),
 ('spark.app.name', 'local'),
 ('spark.driver.maxResultSize', '8g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6')]


<SparkContext master=local[*] appName=local>

**This part we generate the dataframe that part 6.2 will be used:**
* dominant topic per year dataframe;
* average topic weight per year dataframe;
* dominant topic per month dataframe (only for WWI analysis);
* average topic weight per month dataframe (only for WWI analysis);

**After training topic models, we got the topic words list (**`topicKeys.txt`**) and the doc-topic matrix (**`docTopics.txt`**). The doc-topic matrix is a docmument based file which means we could connect the topic weights in the matrix and the metadata (date, region, publisher) in the meta dataset, then the combined dataset could apply to accomplish many data mining or statistical works, for instance, the granularity of time can achieve 1 day, which could perform high accurate time series analsys. Now we load those data to generate dataframe for analysys and visualization.**

## 1 Prepare Dataframe for Train Set

### 1.1 Load data

**Load metadata ("id", "region" and "date"):**

In [2]:
df_meta = load_dataset('meta', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

In [3]:
df_meta.limit(5).toPandas().head()

,id_,region,date
0,1854213,Auckland,1862-06-14
1,1854215,Auckland,1862-06-14
2,1854221,Auckland,1862-06-14
3,1854224,Auckland,1862-07-03
4,1854232,Auckland,1863-08-01


**Load topic words list:**

In [4]:
path = r'../models/train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight_', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)
topic_number = df_topics.count()

In [5]:
print('Shape of dataframe: ({}, {})'.format(topic_number, len(df_topics.columns)))
df_topics.limit(10).toPandas().head(10)

Shape of dataframe: (200, 3)


,topic,weight_,words
0,0,0.00657,train railway leave trains station passengers return dep day tickets service stations express run railways fares spe...
1,1,0.01074,prices advertisements column suit latest ladies suits made street quality goods good wear style special styles fit c...
2,2,0.02215,time good year made great present work years fact number past season week large make place success doubt interest ago
3,3,0.00384,plaintiff defendant claim court costs judgment case witness evidence paid amount pay appeared gave made plaintiffs c...
4,4,0.00901,found death body deceased man inquest dead named evidence witness died suicide morning murder wife returned house ve...
5,5,0.00515,sale harness horses horse draught farm cows stock good cattle mare dairy plough sell cart years spring gelding instr...
6,6,0.00972,motor masterton street car cycle phone cars prices advertisements tyres column price cycles good agents call stock g...
7,7,0.00761,board committee meeting chairman report received messrs letter resolved present held read decided matter reported mo...
8,8,0.00759,goods books fancy christmas stock cases large sets presents xmas assortment cards prices toys variety boxes statione...
9,9,0.00479,hotel proprietor house accommodation rooms hot private visitors mrs day tariff baths moderate telephone good late st...


**Load doc-topic matrix:**

In [6]:
path = r'../models/train/docTopics.txt'

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [7]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (3025602, 202)


,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,1854213,0.000196,0.000321,0.777824,0.000115,0.000269,0.000154,0.000291,0.000227,0.000227,0.000143,0.000254,0.000210,0.000139,0.000235,0.000221,0.000279,0.000245,0.000247,0.000178,0.000305,0.000135,0.090193,0.000215,0.000042,0.000103,0.000260,0.000069,0.000106,0.000294,0.000197,0.000227,0.000176,0.000121,0.031044,0.000048,0.000190,0.000268,0.000177,0.000158,0.000230,0.000169,0.000165,0.000215,0.000315,0.000183,0.000184,0.000214,0.000159,0.000065,0.000088,0.000263,0.000187,0.000139,0.000104,0.000165,0.000172,0.000155,0.000288,...,0.000170,0.000325,0.000338,0.000191,0.000035,0.000223,0.000237,0.000350,0.000084,0.000255,0.000391,0.000196,0.000191,0.000165,0.000213,0.000213,0.000292,0.000288,0.000261,0.000406,0.000239,0.000134,0.000109,0.000175,0.000244,0.000219,0.000119,0.000173,0.000272,0.000128,0.000257,0.000112,0.000072,0.000100,0.000155,0.000189,0.000134,0.000111,0.000168,0.000158,0.000222,0.000211,0.000307,0.000206,0.000256,0.000198,0.000159,0.000438,0.000215,0.000215,0.000269,0.000322,0.000177,0.000282,0.000290,0.000466,0.000188,0.000201,0.000230,0.000148
1,1,1854215,0.000112,0.000184,0.000379,0.000066,0.000154,0.000088,0.000166,0.000130,0.102773,0.000082,0.000146,0.000120,0.000080,0.000135,0.000126,0.000160,0.000140,0.000141,0.000102,0.000174,0.000077,0.000298,0.000123,0.000024,0.000059,0.000149,0.000040,0.000061,0.000168,0.000113,0.000130,0.000101,0.000069,0.034874,0.000028,0.000109,0.000153,0.000101,0.000091,0.000131,0.000097,0.000095,0.000123,0.000180,0.000104,0.000106,0.000123,0.000091,0.000037,0.000050,0.000151,0.000107,0.000080,0.000060,0.000094,0.000098,0.000089,0.000165,...,0.000097,0.000186,0.000193,0.000109,0.000020,0.000128,0.000135,0.000200,0.000048,0.000146,0.000224,0.000112,0.000109,0.000094,0.000122,0.325158,0.000167,0.000165,0.000149,0.000232,0.000137,0.000077,0.000062,0.000100,0.000140,0.000125,0.000068,0.000099,0.000156,0.000073,0.000147,0.000064,0.000041,0.000057,0.000089,0.000108,0.000077,0.000064,0.000096,0.000090,0.000127,0.000121,0.000176,0.000118,0.000147,0.000113,0.000091,0.000251,0.000123,0.000123,0.000154,0.000184,0.051423,0.000161,0.000166,0.000267,0.000108,0.000115,0.000131,0.000085
2,2,1854221,0.000158,0.000259,0.000534,0.000093,0.000217,0.000124,0.000234,0.000184,0.000183,0.000116,0.000205,0.000169,0.000112,0.000190,0.000178,0.000225,0.000198,0.000199,0.000144,0.000246,0.000109,0.000420,0.000173,0.000034,0.000083,0.000210,0.000056,0.000086,0.000237,0.000159,0.000183,0.000142,0.000098,0.025053,0.000039,0.000154,0.000216,0.892676,0.000128,0.000185,0.000137,0.000133,0.000173,0.000254,0.000147,0.000149,0.000173,0.000128,0.000052,0.000071,0.000213,0.000151,0.000112,0.000084,0.000133,0.000139,0.000125,0.000233,...,0.000137,0.000263,0.000273,0.000154,0.000028,0.000180,0.000191,0.000282,0.000068,0.000206,0.000316,0.000158,0.000154,0.000133,0.000172,0.000172,0.000236,0.000232,0.000210,0.024450,0.000193,0.000108,0.000088,0.000141,0.000197,0.000177,0.000096,0.000139,0.000220,0.000103,0.000208,0.000091,0.000058,0.000081,0.000125,0.000153,0.000108,0.000090,0.000136,0.000127,0.000179,0.000170,0.000248,0.000166,0.000207,0.000159,0.000129,0.000354,0.000174,0.000174,0.000217,0.000260,0.000143,0.000228,0.000234,0.000376,0.000152,0.000162,0.000185,0.000120
3,3,1854224,0.000148,0.000242,0.000498,0.000086,0.000203,0.000116,0.000219,0.000171,0.000171,0.000108,0.000191,0.000158,0.000105,0.000177,0.000166,0.000210,0.000185,0.000186,0.000134,0.000229,0.000102,0.000391,0.000161,0.000032,0.000077,0.000196,0.000052,0.000080,0.000221,0.000148,0.000171,0.000133,0.000091,0.000868,0.000036,0.000143,0.000201,0.9

**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to max topic number columns are the weight of each topic per document.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [8]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def add_domtopic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('domtopic',
                        argmax_udf(df.columns[2:])(*df.columns[2:]))
            .withColumn('weight', 
                        F.greatest(*[F.col(x) for x in df.columns[2:-1]])))

# add the df_dominant to doc-topic matrix
df_doctopic = add_domtopic(df_doctopic)

In [9]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.3 Add Metadata Columns

**Here we only add "region" and "date" column as metadata, the accuracy of time could achieve "day" level, using the time series features we could convert "date" to year base or month base etc. depending on the need.**

In [10]:
df_doctopic = (df_doctopic
               .join(df_meta, df_doctopic.id == df_meta.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [11]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.4 Document - Dominant Topics Dataframe

**It is hard to intuitively plot the doc-topic matrix (high dimension), we need to transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [12]:
df_docdomtopic = (df_doctopic
                  .join(df_topics, df_doctopic.domtopic == df_topics.topic)
                  .select(F.col('id'), 
                          F.col('region'), 
                          F.col('year'), 
                          F.col('domtopic'), 
                          F.col('weight'), 
                          F.col('words'))
                  .orderBy('id'))

df_docdomtopic.cache();

In [13]:
print('Shape of dataframe: ({}, {})'.format(df_docdomtopic.count(), len(df_docdomtopic.columns)))

df_docdomtopic.limit(5).toPandas().head()

Shape of dataframe: (3025602, 6)


,id,region,year,domtopic,weight,words
0,1854213,Auckland,1862,2,0.777824,time good year made great present work years fact number past season week large make place success doubt interest ago
1,1854215,Auckland,1862,112,0.393616,advertisements office printing paper column cards published times notice business advertising books post insertion s...
2,1854221,Auckland,1862,37,0.892676,love day life heart thy eyes light long night sweet home great world sun thou beautiful bright man sea beauty
3,1854224,Auckland,1862,37,0.922413,love day life heart thy eyes light long night sweet home great world sun thou beautiful bright man sea beauty
4,1854232,Auckland,1863,12,0.271895,freight apply passage new_zealand company steam passengers london steamers shipping tons wellington agents line stea...


**Save the dataframe for later use:**

In [14]:
path = r'../models/train/domTopics'

df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [15]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could reveal the weight variety of each topic as time goes on, we could take the average weight as features and execute data mining algorithms to find patterns, e.g. correlation between features. The weights of each topic were already scaled to 0-1 by default, so the sum of average weight of each year is 1, we do not need to scale it.**

In [16]:
df_avgweight = (df_doctopic
                .drop('index')
                .drop('id')
                .drop('domtopic')
                .drop('region')
                .drop('weight')
                .groupBy('year')
                .avg()
                .orderBy('year'))

In [17]:
df_avgweight.limit(5).toPandas().head()

,year,avg(0),avg(1),avg(2),avg(3),avg(4),avg(5),avg(6),avg(7),avg(8),avg(9),avg(10),avg(11),avg(12),avg(13),avg(14),avg(15),avg(16),avg(17),avg(18),avg(19),avg(20),avg(21),avg(22),avg(23),avg(24),avg(25),avg(26),avg(27),avg(28),avg(29),avg(30),avg(31),avg(32),avg(33),avg(34),avg(35),avg(36),avg(37),avg(38),avg(39),avg(40),avg(41),avg(42),avg(43),avg(44),avg(45),avg(46),avg(47),avg(48),avg(49),avg(50),avg(51),avg(52),avg(53),avg(54),avg(55),avg(56),avg(57),avg(58),...,avg(140),avg(141),avg(142),avg(143),avg(144),avg(145),avg(146),avg(147),avg(148),avg(149),avg(150),avg(151),avg(152),avg(153),avg(154),avg(155),avg(156),avg(157),avg(158),avg(159),avg(160),avg(161),avg(162),avg(163),avg(164),avg(165),avg(166),avg(167),avg(168),avg(169),avg(170),avg(171),avg(172),avg(173),avg(174),avg(175),avg(176),avg(177),avg(178),avg(179),avg(180),avg(181),avg(182),avg(183),avg(184),avg(185),avg(186),avg(187),avg(188),avg(189),avg(190),avg(191),avg(192),avg(193),avg(194),avg(195),avg(196),avg(197),avg(198),avg(199)
0,1839,0.000022,0.000425,0.000911,0.000013,0.000030,0.000017,0.000032,0.000025,0.000025,0.000016,0.005481,0.000023,0.019074,0.023314,0.000024,0.032158,0.000027,0.019974,0.003910,0.007707,0.000015,0.007991,0.000024,0.000005,0.000011,0.000029,0.000008,0.000012,0.000032,0.000022,0.000025,0.000019,0.000013,0.001686,0.000005,0.000021,0.000029,0.000019,0.000017,0.000025,0.209892,0.000018,0.000024,0.000035,0.000020,0.007599,0.000024,0.000017,0.000007,0.000010,0.000029,0.000021,0.000405,0.001308,0.009868,0.000019,0.000017,0.000032,0.000034,...,0.000019,0.000036,0.000037,0.000021,0.000004,0.000025,0.000026,0.000039,0.000009,0.000028,0.010118,0.003396,0.000021,0.004306,0.000023,0.000023,0.000032,0.000032,0.136809,0.000045,0.000026,0.000015,0.000012,0.000019,0.000027,0.000024,0.000013,0.015223,0.003056,0.000014,0.000028,0.000012,0.000008,0.025350,0.000017,0.000112,0.000015,0.000012,0.029783,0.001577,0.000024,0.006863,0.000034,0.000411,0.000028,0.000022,0.000109,0.000048,0.000024,0.000024,0.010837,0.026408,0.000020,0.000923,0.023499,0.000051,0.007499,0.000022,0.000025,0.000016
1,1840,0.000081,0.000503,0.001999,0.000763,0.004680,0.001690,0.000216,0.001819,0.001664,0.000117,0.001402,0.000229,0.010482,0.007267,0.001201,0.000782,0.001086,0.002136,0.002577,0.021677,0.000285,0.025133,0.002325,0.000017,0.000120,0.003099,0.000028,0.000191,0.002528,0.001410,0.000093,0.003402,0.013162,0.105065,0.000217,0.001906,0.001721,0.000089,0.000471,0.006302,0.056248,0.000068,0.001125,0.001421,0.004782,0.000633,0.003048,0.000065,0.000242,0.000036,0.001449,0.000512,0.000172,0.098555,0.001812,0.000184,0.000128,0.010088,0.001764,...,0.000143,0.002829,0.000777,0.000564,0.000014,0.006599,0.004181,0.000500,0.000114,0.000322,0.001695,0.013306,0.000079,0.000376,0.000150,0.023368,0.000266,0.005717,0.038471,0.000357,0.000228,0.000098,0.000045,0.000835,0.001051,0.000090,0.000068,0.001377,0.000706,0.000118,0.001322,0.000046,0.000030,0.002334,0.002351,0.002608,0.000969,0.000252,0.003852,0.031229,0.000408,0.023364,0.004443,0.000268,0.000856,0.000670,0.000858,0.002600,0.000283,0.000610,0.001118,0.011030,0.001954,0.001597,0.002822,0.001108,0.007900,0.009607,0.000282,0.000061
2,1841,0.000270,0.000333,0.004577,0.000861,0.001153,0.000425,0.000303,0.000698,0.005899,0.000220,0.002391,0.001761,0.010938,0.003140,0.000447,0.001302,0.000389,0.003336,0.002409,0.012347,0.004211,0.016922,0.000690,0.000019,0.000382,0.001464,0.000304,0.000289,0.002560,0.001814,0.000182,0.004298,0.046995,0.007275,0.000035,0.001499,0.000471,0.002161,0.000312,0.002897,0.044117,0.000143,0.001276,0.003230,0.001671,0.002199,0.005198,0.000568,0.000043,0.000164,0.001202,0.000159,0.000383,0.073176,0.001844,0.000078,0.000071,0.002074,0.001622,...,0.000078,0.000325,0.004013,0.000300,0.000016,0.001277,0.002131,0.000712,0.000067,0.000205,0.002044,0.006398,0.000896,0.000227,0.000356,0.004799,0.000206,0.005139,0.043389,0.000520,0.001673,0.000085,0.000050,0.005762,0.000430,0.000537,0.000138,0.001076,0.000491,0.003100,0.001666,0.0

**check years are identical with dataset:**

In [18]:
year_doct = list(df_doctopic.select('year').distinct().orderBy('year').rdd.map(lambda r: r[0]).collect())
year_avgw = list(df_avgweight.select('year').rdd.map(lambda r: r[0]).collect())
pd.DataFrame({'yearDocTopic':year_doct, 'yearAvgWeight':year_avgw}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
yearAvgWeight,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945
yearDocTopic,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945


**The dimension of the avrage weight dataframe is small (topic_n\*year_n), so we directly save the dataframe using Pandas:**

In [19]:
path = r'../models/train/avgWeight.csv'

df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

## 2 Prepare Dataframe for Subset

**The process for subset is the same with for full dataset, we wrapped the process to a function and call it for each subset.**

### 2.1 By Range of Time

**The tiem range is changed to short time, so we need to generate new weight for topic list in** `topicKeys.csv`**.**

In [20]:
path = r'../models/wwi/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_wwi = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_wwi = df_topics_wwi[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_wwi.shape)
#df_topics_wwi.head()

Shape of dataframe: (200, 3)


In [21]:
path = r'../models/wwi/topicKeys.csv'
df_topics_wwi.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except WWI).**

In [22]:
path = r'../models/train/docTopics.txt'

START = '1912-01-01'
END = '1921-12-31'

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter((df_meta['date'] < START) | (df_meta['date'] > END))
df_doctopic = (df_doctopic
               .join(df_sub, df_doctopic.id == df_sub.id_)
               .drop('index').drop('id').drop('id_').drop('region').drop('date'))
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [23]:
path = r'../models/wwi/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [24]:
path = r'../models/wwi/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'month')

df_docdomtopic.cache();

**Save dataframes:**

In [25]:
path = r'../models/wwi/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/wwi/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist();

**Convert multi files to a csv file:**

In [26]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.2 By Region

#### 2.2.1 Otago

**Generate new weight for regions.**

In [33]:
path = r'../models/otago/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


In [34]:
path = r'../models/otago/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except The Regions List).**

In [35]:
path = r'../models/train/docTopics.txt'
#regions = ['Otago', 'Canterbury', 'Manawatu-Wanganui', 'Wellington']
regions = ['Otago']

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('region').isin(regions) == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [36]:
path = r'../models/otago/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [37]:
path = r'../models/otago/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache();

**Save dataframes:**

In [38]:
path = r'../models/otago/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/otago/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist();

**Convert multi files to a csv file:**

In [39]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

#### 2.2.2 Canterbury

**Generate new weight for regions.**

In [40]:
path = r'../models/canterbury/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


In [41]:
path = r'../models/canterbury/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except The Regions List).**

In [42]:
path = r'../models/train/docTopics.txt'
regions = ['Canterbury']

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('region').isin(regions) == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [43]:
path = r'../models/canterbury/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [44]:
path = r'../models/canterbury/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache();

**Save dataframes:**

In [45]:
path = r'../models/canterbury/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/canterbury/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist();

**Convert multi files to a csv file:**

In [46]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

#### 2.2.3 Manawatu-Wanganui

**Generate new weight for regions.**

In [47]:
path = r'../models/manawatu-wanganui/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


In [48]:
path = r'../models/manawatu-wanganui/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except The Regions List).**

In [49]:
path = r'../models/train/docTopics.txt'
regions = ['Manawatu-Wanganui']

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('region').isin(regions) == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [50]:
path = r'../models/manawatu-wanganui/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [51]:
path = r'../models/manawatu-wanganui/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache();

**Save dataframes:**

In [52]:
path = r'../models/manawatu-wanganui/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/manawatu-wanganui/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist();

**Convert multi files to a csv file:**

In [53]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

#### 2.2.4 Wellington

**Generate new weight for regions.**

In [54]:
path = r'../models/wellington/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


In [55]:
path = r'../models/wellington/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except The Regions List).**

In [56]:
path = r'../models/train/docTopics.txt'
regions = ['Wellington']

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('region').isin(regions) == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [57]:
path = r'../models/wellington/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [58]:
path = r'../models/wellington/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache();

**Save dataframes:**

In [59]:
path = r'../models/wellington/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/wellington/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist();

**Convert multi files to a csv file:**

In [60]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.3 By Label

**Generate new weight for ADs.**

In [61]:
path = r'../models/ads/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_ads = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_ads = df_topics_ads[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_ads.shape)
#df_topics_ads.head()

Shape of dataframe: (200, 3)


In [62]:
path = r'../models/ads/topicKeys.csv'
df_topics_ads.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate new weight for others (except ADs).**

In [63]:
path = r'../models/train/docTopics.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_sub = df_meta.filter(F.col('ads') == False)
df_doctopic = df_doctopic.join(df_sub, df_doctopic.id == df_sub.id_).drop('index').drop('id').drop('id_').drop('region').drop('date')
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_others = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_others = df_topics_others[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_others.shape)
#df_topics_others.head()

Shape of dataframe: (200, 3)


In [64]:
path = r'../models/ads/topicKeysOthers.csv'
df_topics_others.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [65]:
path = r'../models/ads/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Save dataframes:**

In [66]:
path = r'../models/ads/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/ads/domTopics'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

**Convert multi files to a csv file:**

In [67]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

----

In [68]:
sc.stop()
gc.collect()

144